# <span style="color: #00ab9c">Part 2:</span> Limitations of sklearn's NMF Library

## Loading movie ratings from homework 3

In [41]:
# I used some of the code below from the HW3 assignment
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple

MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

In [42]:
print(data.train.head())

    uID   mID  rating
0   744  1210       5
1  3040  1584       4
2  1451  1293       5
3  5455  3176       2
4  2507  3074       5


In [45]:
rating_matrix = data.train.pivot(index='uID', columns='mID', values='rating')
print(rating_matrix)

mID   1     2     3     4     5     6     7     8     9     10    ...  3943  \
uID                                                               ...         
1      5.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
2      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
3      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
4      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
5      NaN   NaN   NaN   NaN   NaN   2.0   NaN   NaN   NaN   NaN  ...   NaN   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
6036   NaN   NaN   NaN   2.0   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
6037   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
6038   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
6039   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
6040   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN

In [46]:
rating_matrix_zero = rating_matrix.copy()
rating_matrix_zero[np.isnan(rating_matrix_zero)] = 0
print(rating_matrix_zero)

mID   1     2     3     4     5     6     7     8     9     10    ...  3943  \
uID                                                               ...         
1      5.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5      0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
6036   0.0   0.0   0.0   2.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6037   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6038   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6039   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6040   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0

In [47]:
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=5, random_state=40)
W = nmf_model.fit_transform(rating_matrix_zero)
H = nmf_model.components_

rating_matrix_hat = np.dot(W, H)
rating_matrix_reconstructed = pd.DataFrame(rating_matrix_hat, index=rating_matrix.index, columns=rating_matrix.columns)

rating_matrix_filled = rating_matrix.copy()

mask = rating_matrix.isna()
rating_matrix_filled[mask] = rating_matrix_reconstructed[mask]

print(rating_matrix_reconstructed)
print(rating_matrix_filled)


mID       1         2         3         4         5         6         7     \
uID                                                                          
1     0.829939  0.218400  0.180189  0.084691  0.172853  0.105464  0.293883   
2     0.492053  0.347704  0.145447  0.017005  0.065835  0.928003  0.060594   
3     0.548136  0.160800  0.082487  0.032135  0.068807  0.180249  0.110412   
4     0.284615  0.056549  0.000000  0.000000  0.000000  0.013092  0.000000   
5     1.152778  0.048671  0.022790  0.044850  0.010316  0.809842  0.056227   
...        ...       ...       ...       ...       ...       ...       ...   
6036  2.901329  0.666377  0.377042  0.166702  0.299311  1.534222  0.501944   
6037  0.874156  0.093107  0.000000  0.010125  0.000000  0.252311  0.011216   
6038  0.176853  0.045310  0.029047  0.013138  0.027865  0.014096  0.046805   
6039  0.605095  0.163958  0.124029  0.054080  0.118980  0.047469  0.197619   
6040  0.938475  0.044392  0.010392  0.026378  0.009969  0.437410

## Measuring RSME

In [48]:
from sklearn.metrics import root_mean_squared_error

def get_predicted(row):
    try:
        return rating_matrix_reconstructed.at[row['uID'], row['mID']]
    except KeyError:
        return np.nan
        
test['predicted'] = test.apply(get_predicted, axis=1)
test_clean = test.dropna(subset=['predicted'])

rmse = root_mean_squared_error(test_clean['rating'], test_clean['predicted'])
print(f'RMSE: {rmse}')

RMSE: 2.9914125530578106


## Addressing lack of performance

I used NMF to fill in ratings for movies that users did not rate, but, as you see and as was predicted, it did not work well compared to the methods from the homework assignment, which had RMSE values around 1. One reason for this is that NMF does not handle data sparsity well, which the ratings matrix had (many movies missing ratings). Another reason could be NMF's reliance on latent factors where the other methods are able to find patterns and relationships within the data.

NMF might be able to be improved for this use case by using it in conjunction with a similarity based method. This could involved using NMF to find latent factors and then proceeding with a similarity based method. We could combine one of the baseline approaches like imputing to user average prior to using NMF to incorporate missing patterns and relationships.